In [19]:
from interchange.persistence.file import FileStorage
from interchange.visa import transform, extract, clean, calculate, interchange, store

layer = FileStorage.Layer

In [20]:
client_id = "SBSA"
file_id = "B6781ADDCFE0CD800BFA2968A6ED2816"

In [21]:
origin_layer = layer.STAGING
target_layer = layer.STAGING
client_id = client_id
file_id = file_id
transactions_subdir = "300-SMS_CLN_MESSAGES"
calculated_subdir = "400-SMS_CAL_MESSAGES"
target_subdir = "500-SMS_ITX_MESSAGES"

In [22]:
from datetime import date

import numpy as np
import pandas as pd

from interchange.logs.logger import Logger
from interchange.persistence.database import Database
from interchange.persistence.file import FileStorage


log = Logger(__name__)
fs = FileStorage()

# FUNCIONES

In [23]:
def _get_file_data(client_id: str, file_id: str) -> pd.Series:
    """
    Get key metadata associated to an interchange file.
    """
    db = Database()
    fd = db.read_records(
        table_name="file_control",
        fields=[
            "brand_id",
            "file_type",
            "file_processing_date",
        ],
        where={"client_id": client_id, "file_id": file_id},
    )
    fd["file_processing_date"] = pd.to_datetime(
        fd["file_processing_date"], format="%Y-%m-%d"
    ).dt.date
    return fd.iloc[0]

In [24]:
def _get_visa_rule_definitions(file_date: date, type_record: str) -> pd.DataFrame:
    """
    Get Visa's interchange rule assignment criteria for the file's processing date.
    """
    db = Database()
    df = db.read_records(
        table_name="visa_rules_2",
        fields=[
            "region_country_code",
            "valid_from",
            "valid_until",
            "intelica_id",
            "fee_descriptor",
            "fee_currency",
            "fee_variable",
            "fee_fixed",
            "fee_min",
            "fee_cap",
            "business_mode",
            "issuer_country",
            "issuer_region",
            "technology_indicator",
            "product_id",
            "fast_funds",
            "travel_indicator",
            "b2b_program_id",
            "account_funding_source",
            "nnss_indicator",
            "product_subtype",
            "transaction_code",
            "transaction_code_qualifier",
            "issuer_bin_8",
            "acquirer_bin",
            "acquirer_business_id",
            "transaction_amount_currency",
            "transaction_amount",
            "acquirer_country",
            "acquirer_region",
            "merchant_country_code",
            "merchant_country_region",
            "merchant_category_code",
            "requested_payment_service",
            "usage_code",
            "authorization_characteristics_indicator",
            "authorization_code",
            "pos_terminal_capability",
            "cardholder_id_method",
            "pos_entry_mode",
            "timeliness",
            "reimbursement_attribute",
            "special_condition_indicator",
            "fee_program_indicator",
            "moto_eci_indicator",
            "acceptance_terminal_indicator",
            "prepaid_card_indicator",
            "pos_environment_code",
            "business_format_code",
            "business_application_id",
            "type_purchase",
            "network_identification_code",
            "message_reason_code",
            "surcharge_amount",
            "authorized_amount",
            "authorization_response_code",
            "merchant_verification_value",
            "dynamic_currency_conversion_indicator",
            "cvv2_result_code",
            "national_tax_indicator",
            "merchant_vat",
            "summary_commodity",
            "processing_code_transaction_type",
            "point_of_service_condition_code",
        ],
    )
    int_cols = ["intelica_id"]
    numeric_cols = ["fee_variable", "fee_fixed", "fee_min", "fee_cap"]
    date_cols = ["valid_from", "valid_until"]
    df[int_cols] = df[int_cols].apply(
        pd.to_numeric, downcast="integer", errors="coerce"
    )
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
    for col in date_cols:
        df[col] = pd.to_datetime(
            df[col].str.slice(0, 10), format="%Y-%m-%d", errors="coerce"
        ).dt.date
    df[date_cols] = df[date_cols].fillna(date.today())
    df_valid = df[(file_date >= df["valid_from"]) & (file_date <= df["valid_until"])]
    df_valid = df_valid.sort_values(["region_country_code", "intelica_id"])
    match type_record:
        case "draft":
            df_valid.drop(
                columns=[
                    "acquirer_country",
                    "acquirer_region",
                    "processing_code_transaction_type",
                    "point_of_service_condition_code",
                ],
                inplace=True,
            )
            df_valid.rename(
                columns={
                    "account_funding_source": "funding_source",
                    "acquirer_bin": "account_reference_number_acquiring_identifier",
                    "cvv2_result_code": "cvv_result_code",
                    "dynamic_currency_conversion_indicator": "dcc_indicator",
                    "merchant_country_code": "jurisdiction_country",
                    "merchant_country_region": "jurisdiction_region",
                    "merchant_vat": "merchant_vat_registration_number",
                    "moto_eci_indicator": "moto_ec_indicator",
                    "national_tax_indicator": "national_tax_included",
                    "pos_environment_code": "pos_environment",
                    "pos_terminal_capability": "pos_terminal_capacity",
                    "special_condition_indicator": "special_condition_indicator_merchant_draft_indicator",
                    "summary_commodity": "summary_commodity_code",
                    "transaction_amount": "source_amount",
                    "transaction_code_qualifier": "draft_code_qualifier_0",
                    "transaction_code": "draft_code",
                    "type_purchase": "type_of_purchase",
                },
                inplace=True,
            )
            return df_valid
        case "sms":
            df_valid.drop(
                columns=[
                    "acquirer_country",
                    "acquirer_region",
                    "authorized_amount",
                    "business_format_code",
                    "merchant_vat",
                    "national_tax_indicator",
                    "prepaid_card_indicator",
                    "summary_commodity",
                    "transaction_amount_currency",
                    "transaction_code_qualifier",
                    "type_purchase",
                ],
                inplace=True,
            )
            df_valid.rename(
                columns={
                    "account_funding_source": "funding_source",
                    "acceptance_terminal_indicator": "pos_terminal_type",
                    "acquirer_business_id": "acquirer_business_id_sms",
                    "authorization_characteristics_indicator": "authorization_characteristics_indicator_sms",
                    "authorization_code": "authorization_code_valid",
                    "authorization_response_code": "response_code",
                    "business_application_id": "business_application_identifier",
                    "cardholder_id_method": "customer_identification_method",
                    "cvv2_result_code": "cvv_result_code_sms",
                    "dynamic_currency_conversion_indicator": "dcc_indicator_sms",
                    "fee_program_indicator": "fee_program_indicator_sms",
                    "merchant_category_code": "merchant's_type",
                    "merchant_country_code": "jurisdiction_country",
                    "merchant_country_region": "jurisdiction_region",
                    "merchant_verification_value": "mvv_code",
                    "message_reason_code": "message_reason_code_sms",
                    "moto_eci_indicator": "mail_telephone_or_electronic_commerce_indicator",
                    "network_identification_code": "network_id",
                    "point_of_service_condition_code": "pos_condition_code",
                    "pos_environment_code": "recurring_payment_indicator_flag",
                    "pos_entry_mode": "pos_entry_mode_sms",
                    "pos_terminal_capability": "pos_terminal_entry_capability",
                    "reimbursement_attribute": "reimbursement_attribute_sms",
                    "special_condition_indicator": "chargeback_special_condition_merchant_indicator",
                    "summary_commodity": "summary_commodity_code",
                    "surcharge_amount": "surcharge_amount_sms",
                    "transaction_amount": "source_amount",
                    "transaction_code_qualifier": "draft_code_qualifier_0",
                    "transaction_code": "transaction_code_sms",
                    "usage_code": "usage_code_sms",
                },
                inplace=True,
            )
            return df_valid
        case _:
            raise NotImplementedError

In [25]:
def _get_exchange_rates(file_date: date, brand: str) -> pd.DataFrame:
    """
    Get the exchange rates valid for the file's processing date and brand.
    """
    date_string = file_date.strftime("%Y-%m-%d")
    db = Database()
    df = db.read_records(
        table_name="exchange_rate",
        fields=[
            "currency_from",
            "currency_to",
            "currency_from_code",
            "currency_to_code",
            "exchange_value",
        ],
        where={"brand": brand, "rate_date": date_string},
    )
    df["exchange_value"] = df["exchange_value"].apply(pd.to_numeric, errors="coerce")
    return df


In [26]:
def _apply_condition_default(
    condition_name: str,
    condition_value: str,
    batch: pd.DataFrame,
    column_group_space: list[str],
) -> pd.DataFrame:
    """
    Checks conditions that have a specific value.
    """
    batch = batch.copy()
    condition_value = condition_value.strip().upper()
    condition_value = condition_value.replace("SPACE", " ")
    value_list = condition_value.split(",")
    valid_values = []
    not_valid_values = []
    for value in value_list:
        not_keyword_flag = False
        if "NOT:" in value:
            value = value.replace("NOT:", "")
            not_keyword_flag = True
        filled_range = []
        if "-" in value:
            range_low, range_high = value.split("-", maxsplit=1)
            filled_range = list(range(int(range_low), int(range_high) + 1))
            filled_range = list(map(str, filled_range))
        reformatted_values = filled_range or [value]
        match not_keyword_flag:
            case False:
                valid_values.extend(reformatted_values)
            case True:
                not_valid_values.extend(reformatted_values)

    temp_col = batch[condition_name]
    if condition_name in column_group_space:
        batch["_normalized"] = temp_col.astype(str)
    else:
        temp_col = temp_col.astype(str).str.strip()
        temp_col = temp_col.mask(temp_col.str.len() == 0, "BLANK")
        batch["_normalized"] = temp_col

    filter = batch
    if valid_values:
        filter = filter[filter["_normalized"].isin(valid_values)]
    if not_valid_values:
        filter = filter[~filter["_normalized"].isin(not_valid_values)]
    filter = filter.drop(columns="_normalized")

    return filter

In [27]:
def _apply_condition_greater_less(
    condition_name: str, condition_value: str, batch: pd.DataFrame
) -> pd.DataFrame:
    """
    Check numeric conditions where a value falls in a specified range.
    """
    if any(x in condition_value for x in ["<", ">", "="]):
        query_condition = f"{condition_name} " + condition_value.replace(
            "<=", "<= "
        ).replace(">=", ">= ").replace(">", "> ").replace("<", "< ")

        filter = batch.query(query_condition)
    elif any(x in condition_value for x in ["BETWEEN", "AND"]):
        range_low, range_high = list(
            map(
                float,
                condition_value.replace(" ", "")
                .replace("BETWEEN", "")
                .split("AND", maxsplit=1),
            )
        )
        filter = batch[
            batch[condition_name]
            .astype(float)
            .between(range_low, range_high, inclusive="both")
        ]
    elif condition_value.replace(".", "", 1).isdigit():
        numeric_value = float(condition_value)
        filter = batch[batch[condition_name].astype(float) == numeric_value]
    else:
        raise ValueError

    return filter

In [28]:
def _apply_condition_amount_currency(
    condition_name: str, string_range: str, batch: pd.DataFrame, rates: pd.DataFrame
) -> pd.DataFrame:
    """
    Check currency amount conditions where a value falls in a specified range.
    """
    condition_target_fields = {
        "source_amount": "source_currency_code_alphabetic",
    }
    target_currency, string_range = string_range.split(",", maxsplit=1)
    target_rates = rates[rates["currency_to"] == target_currency]
    filter = pd.merge(
        left=batch,
        right=target_rates[["currency_from", "exchange_value"]],
        how="left",
        left_on=condition_target_fields[condition_name],
        right_on="currency_from",
    )
    filter.loc[
        filter[condition_target_fields[condition_name]] == target_currency,
        "exchange_value",
    ] = 1
    filter["comparison_value"] = filter[condition_name] * filter["exchange_value"]

    if any(x in string_range for x in ["<", ">", "="]):
        query_condition = f"comparison_value " + string_range.replace(
            "<=", "<= "
        ).replace(">=", ">= ").replace(">", "> ").replace("<", "< ")

        filter = filter.query(query_condition)
    elif any(x in string_range for x in ["BETWEEN", "AND"]):
        range_low, range_high = list(
            map(
                float,
                string_range.replace(" ", "")
                .replace("BETWEEN", "")
                .split("AND", maxsplit=1),
            )
        )
        filter = filter[
            filter["comparison_value"].between(range_low, range_high, inclusive="both")
        ]
    else:
        raise ValueError

    return filter

In [29]:
def _apply_condition(
    condition_name: str, condition_value: str, batch: pd.DataFrame, rates: pd.DataFrame
) -> pd.DataFrame:
    """
    Clean, check and apply condition to a batch of transactions.
    """
    # If there is no condition, return the same batch of transactions.
    condition_value = condition_value.replace(" ", "").upper()
    if condition_value in ("", "NAN", "NONE"):
        return batch
    # Otherwise, evaluate the condition.
    column_group_greater_less = [
        "surcharge_amount",
        "surcharge_amount_sms",
        "timeliness",
    ]
    column_group_amount_currency = [
        "source_amount",
    ]
    column_group_space = [
        "nnss_indicator",
        "cardholder_id_method",
        "moto_eci_indicator",
        "acceptance_terminal_indicator",
        "merchant_vat",
    ]

    match condition_name:
        case name if name in column_group_greater_less:
            result = _apply_condition_greater_less(
                condition_name, condition_value, batch
            )
        case name if name in column_group_amount_currency:
            result = _apply_condition_amount_currency(
                condition_name, condition_value, batch, rates
            )
        case _:
            result = _apply_condition_default(
                condition_name, condition_value, batch, column_group_space
            )

    return result

In [30]:
def _evaluate_interchange_fees(
    transactions: pd.DataFrame,
    rules: pd.DataFrame,
    rates: pd.DataFrame,
) -> pd.DataFrame:
    """
    Evaluate interchange fee criteria for a dataset of transactions.
    """
    # Filter rule definitions to only jurisdictions present in data.
    jurisdiction_list = transactions["jurisdiction_assigned"].unique()
    rules_to_evaluate = rules[rules["region_country_code"].isin(jurisdiction_list)]
    # Initialize rule identifier fields.
    transactions["interchange_region_country_code"] = ""
    transactions["interchange_intelica_id"] = -1
    transactions["interchange_fee_descriptor"] = ""
    transactions["interchange_fee_currency"] = ""
    transactions["interchange_fee_variable"] = 0.0
    transactions["interchange_fee_fixed"] = 0.0
    transactions["interchange_fee_min"] = 0.0
    transactions["interchange_fee_cap"] = 0.0
    # Iterate through each rule definition.
    conditions_to_skip = [
        "region_country_code",
        "valid_from",
        "valid_until",
        "intelica_id",
        "fee_descriptor",
        "fee_currency",
        "fee_variable",
        "fee_fixed",
        "fee_min",
        "fee_cap",
    ]
    update_columns = [
        "region_country_code",
        "intelica_id",
        "fee_descriptor",
        "fee_currency",
        "fee_variable",
        "fee_fixed",
        "fee_min",
        "fee_cap",
    ]
    for _, rule in rules_to_evaluate.iterrows():
        # Step 1: Filter unprocessed transactions and decide to break, skip or evaluate.
        next_batch = transactions[
            (transactions["interchange_intelica_id"] == -1)
            & (transactions["jurisdiction_assigned"] == rule["region_country_code"])
        ]
        if next_batch.empty:
            continue
        # Step 2: Iterate through each condition in the rule and apply its condition.
        conditions = [
            str(cond_name)
            for cond_name in rule.index.to_list()
            if cond_name not in conditions_to_skip and rule[cond_name] != ""
        ]
        for condition in conditions:
            next_batch = _apply_condition(condition, rule[condition], next_batch, rates)
            if next_batch.empty:
                break
        # Step 3: Update transaction table with batch results.
        if not next_batch.empty:
            for column in update_columns:
                next_batch.loc[:, f"interchange_{column}"] = rule[column]
            transactions.update(
                next_batch[[f"interchange_{c}" for c in update_columns]]
            )

    columns_to_return = [f"interchange_{c}" for c in update_columns]
    columns_to_return = [
        "source_currency_code_alphabetic",
        "source_amount",
    ] + columns_to_return
    return transactions[columns_to_return]

In [31]:
def _calculate_interchange_fees(
    fee_parameters: pd.DataFrame, rates: pd.DataFrame
) -> pd.DataFrame:
    """
    Calculate interchange fee amounts depending on the assigned fee parameters.
    """
    stage = pd.merge(
        left=fee_parameters,
        right=rates[["currency_from", "currency_to", "exchange_value"]],
        how="left",
        left_on=["source_currency_code_alphabetic", "interchange_fee_currency"],
        right_on=["currency_from", "currency_to"],
    )
    stage.loc[
        stage["source_currency_code_alphabetic"] == stage["interchange_fee_currency"],
        "exchange_value",
    ] = 1
    stage.drop(columns=["currency_from", "currency_to"], inplace=True)

    stage["interchange_fee_variable"] = stage["interchange_fee_variable"].fillna(0)
    cur_cols = ["interchange_fee_fixed", "interchange_fee_min", "interchange_fee_cap"]
    stage[cur_cols] = stage[cur_cols].replace(to_replace=0, value=np.nan)
    for col in cur_cols:
        stage[f"{col}_source"] = stage[col] * stage["exchange_value"]
        match col:
            case "interchange_fee_fixed":
                stage[f"{col}_source"] = stage[f"{col}_source"].fillna(0)
            case "interchange_fee_min":
                stage[f"{col}_source"] = stage[f"{col}_source"].fillna(-np.inf)
            case "interchange_fee_cap":
                stage[f"{col}_source"] = stage[f"{col}_source"].fillna(+np.inf)

    stage["interchange_fee_amount"] = (
        stage["source_amount"] * stage["interchange_fee_variable"]
        + stage["interchange_fee_fixed_source"]
    )
    stage["interchange_fee_amount"] = stage[
        ["interchange_fee_amount", "interchange_fee_min_source"]
    ].max(axis=1)
    stage["interchange_fee_amount"] = stage[
        ["interchange_fee_amount", "interchange_fee_cap_source"]
    ].min(axis=1)

    result = stage.drop(columns=["source_currency_code_alphabetic", "source_amount"])
    return result

# MAIN

In [32]:
transactions = fs.read_parquet(
    origin_layer,
    client_id,
    file_id,
    subdir=transactions_subdir,
)

2025-12-02 11:55:56,105 :: PID 16544 :: TID 28624 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database
2025-12-02 11:55:56,105 :: PID 16544 :: TID 28624 :: database.read_records :: Line 138 :: DEBUG :: Attempting to execute SELECT SQL statement


2025-12-02 11:55:56,115 :: PID 16544 :: TID 28624 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully
2025-12-02 11:55:56,158 :: PID 16544 :: TID 28624 :: database._close_connection :: Line 44 :: DEBUG :: Closed connection to SQLite database


In [33]:
calculated = fs.read_parquet(
    origin_layer,
    client_id,
    file_id,
    subdir=calculated_subdir,
)

2025-12-02 11:55:57,478 :: PID 16544 :: TID 28624 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database
2025-12-02 11:55:57,481 :: PID 16544 :: TID 28624 :: database.read_records :: Line 138 :: DEBUG :: Attempting to execute SELECT SQL statement
2025-12-02 11:55:57,485 :: PID 16544 :: TID 28624 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully
2025-12-02 11:55:57,487 :: PID 16544 :: TID 28624 :: database._close_connection :: Line 44 :: DEBUG :: Closed connection to SQLite database


In [34]:
merged_data = transactions.join(calculated, how="left", lsuffix="_sms")

In [35]:
file_data = _get_file_data(client_id, file_id)
rules_data = _get_visa_rule_definitions(
    file_data["file_processing_date"], type_record="sms"
)


2025-12-02 11:56:03,067 :: PID 16544 :: TID 28624 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database
2025-12-02 11:56:03,067 :: PID 16544 :: TID 28624 :: database.read_records :: Line 138 :: DEBUG :: Attempting to execute SELECT SQL statement
2025-12-02 11:56:03,067 :: PID 16544 :: TID 28624 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully
2025-12-02 11:56:03,098 :: PID 16544 :: TID 28624 :: database._close_connection :: Line 44 :: DEBUG :: Closed connection to SQLite database
2025-12-02 11:56:03,102 :: PID 16544 :: TID 28624 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database
2025-12-02 11:56:03,103 :: PID 16544 :: TID 28624 :: database.read_records :: Line 138 :: DEBUG :: Attempting to execute SELECT SQL statement
2025-12-02 11:56:03,105 :: PID 16544 :: TID 28624 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully
2025-12-02 11:56:03,159 :: PID 16544 :: TID 28624 

In [36]:
rules_data.columns.tolist()

['region_country_code',
 'valid_from',
 'valid_until',
 'intelica_id',
 'fee_descriptor',
 'fee_currency',
 'fee_variable',
 'fee_fixed',
 'fee_min',
 'fee_cap',
 'business_mode',
 'issuer_country',
 'issuer_region',
 'technology_indicator',
 'product_id',
 'fast_funds',
 'travel_indicator',
 'b2b_program_id',
 'funding_source',
 'nnss_indicator',
 'product_subtype',
 'transaction_code_sms',
 'issuer_bin_8',
 'acquirer_bin',
 'acquirer_business_id_sms',
 'source_amount',
 'jurisdiction_country',
 'jurisdiction_region',
 "merchant's_type",
 'requested_payment_service',
 'usage_code_sms',
 'authorization_characteristics_indicator_sms',
 'authorization_code_valid',
 'pos_terminal_entry_capability',
 'customer_identification_method',
 'pos_entry_mode_sms',
 'timeliness',
 'reimbursement_attribute_sms',
 'chargeback_special_condition_merchant_indicator',
 'fee_program_indicator_sms',
 'mail_telephone_or_electronic_commerce_indicator',
 'pos_terminal_type',
 'recurring_payment_indicator_flag

In [37]:
rates = _get_exchange_rates(file_data["file_processing_date"], brand="VISA")

2025-12-02 11:56:47,239 :: PID 16544 :: TID 28624 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database
2025-12-02 11:56:47,241 :: PID 16544 :: TID 28624 :: database.read_records :: Line 138 :: DEBUG :: Attempting to execute SELECT SQL statement
2025-12-02 11:56:47,719 :: PID 16544 :: TID 28624 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully
2025-12-02 11:56:47,873 :: PID 16544 :: TID 28624 :: database._close_connection :: Line 44 :: DEBUG :: Closed connection to SQLite database


In [42]:
fee_parameters = _evaluate_interchange_fees(merged_data, rules_data, rates)

In [43]:
interchange_df = _calculate_interchange_fees(fee_parameters, rates)

In [44]:
fs.write_parquet(interchange_df, target_layer, client_id, file_id, subdir=target_subdir)

2025-12-01 13:54:21,694 :: PID 20676 :: TID 29572 :: file.write_parquet :: Line 128 :: DEBUG :: Writing SBSA file B6781ADDCFE0CD800BFA2968A6ED2816 to parquet
2025-12-01 13:54:21,699 :: PID 20676 :: TID 29572 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database
2025-12-01 13:54:21,701 :: PID 20676 :: TID 29572 :: database.read_records :: Line 138 :: DEBUG :: Attempting to execute SELECT SQL statement
2025-12-01 13:54:21,702 :: PID 20676 :: TID 29572 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully
2025-12-01 13:54:21,706 :: PID 20676 :: TID 29572 :: database._close_connection :: Line 44 :: DEBUG :: Closed connection to SQLite database


In [38]:
transactions = merged_data.copy()
rules = rules_data.copy()


In [39]:
jurisdiction_list = transactions["jurisdiction_assigned"].unique()
rules_to_evaluate = rules[rules["region_country_code"].isin(jurisdiction_list)]

In [40]:
transactions["interchange_region_country_code"] = ""
transactions["interchange_intelica_id"] = -1
transactions["interchange_fee_descriptor"] = ""
transactions["interchange_fee_currency"] = ""
transactions["interchange_fee_variable"] = 0.0
transactions["interchange_fee_fixed"] = 0.0
transactions["interchange_fee_min"] = 0.0
transactions["interchange_fee_cap"] = 0.0

In [41]:
# Iterate through each rule definition.
conditions_to_skip = [
    "region_country_code",
    "valid_from",
    "valid_until",
    "intelica_id",
    "fee_descriptor",
    "fee_currency",
    "fee_variable",
    "fee_fixed",
    "fee_min",
    "fee_cap",
]
update_columns = [
    "region_country_code",
    "intelica_id",
    "fee_descriptor",
    "fee_currency",
    "fee_variable",
    "fee_fixed",
    "fee_min",
    "fee_cap",
]

In [43]:
conditions_to_skip

['region_country_code',
 'valid_from',
 'valid_until',
 'intelica_id',
 'fee_descriptor',
 'fee_currency',
 'fee_variable',
 'fee_fixed',
 'fee_min',
 'fee_cap']

In [45]:
for _, rule in rules_to_evaluate.iterrows():
        # Step 1: Filter unprocessed transactions and decide to break, skip or evaluate.
        next_batch = transactions[
            (transactions["interchange_intelica_id"] == -1)
            & (transactions["jurisdiction_assigned"] == rule["region_country_code"])
        ]
        if next_batch.empty:
            continue
        # Step 2: Iterate through each condition in the rule and apply its condition.
        conditions = [
            str(cond_name)
            for cond_name in rule.index.to_list()
            if cond_name not in conditions_to_skip and rule[cond_name] != ""
        ]
        for condition in conditions:
            next_batch = _apply_condition(condition, rule[condition], next_batch, rates)
            if next_batch.empty:
                break
        # Step 3: Update transaction table with batch results.
        if not next_batch.empty:
            for column in update_columns:
                next_batch.loc[:, f"interchange_{column}"] = rule[column]
            transactions.update(
                next_batch[[f"interchange_{c}" for c in update_columns]]
            )

In [ ]:
columns_to_return = [f"interchange_{c}" for c in update_columns]

In [ ]:
columns_to_return = [
    "source_currency_code_alphabetic",
    "source_amount",
] + columns_to_return

In [ ]:
transactions[columns_to_return]